<a href="https://colab.research.google.com/github/shiwangi27/googlecolab/blob/main/transformer_pytorch_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch for building and training Transformer NLP models from scratch. 

In [ ]:
import math
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F 

In [ ]:
class BertConfig:
  pass

In [ ]:
class SelfAttention(nn.Module):
  """
  Multi-head Self-attention is the learnt subspace representation of a sequence
  through linear projections of 
  attention = softmax (k . q / √d) * v 
  """
  def __init__(self, embed_dim: int, 
               num_heads: int, 
               activation: nn.functional = F.relu, 
               bias: bool = True, 
               mask: bool = False) -> Tensor:
    super(SelfAttention, self).__init__()

    self.num_heads = num_heads
    self.embed_dim = embed_dim
    self.activation = activation
    self.bias = bias

    self.q = nn.Linear(embed_dim, embed_dim)
    self.k = nn.Linear(embed_dim, embed_dim)
    self.v = nn.Linear(embed_dim, embed_dim)

    self.out_projection = nn.Linear(embedding_dim, embedding_dim)
  
  def _reshape_to_subspace(x):
    batch_size, seq_len, embed_dim = x.size()
    subspace_dim = embed_sim // self.num_heads
    new_batch_size = batch_size * self.num_heads

    return x.reshape(batch_size, seq_len, self.num_heads, subspace_dim)\
            .permute((0, 2, 1, 3))\
            .reshape(new_batch_size, seq_len, subspace_dim)

  def _reshape_from_subspace(x):
    new_batch_size, seq_len, subspace_dim = x.size()
    embed_dim = subspace_dim * self.num_heads
    batch_size = new_batch_size // self.num_heads

    return x.reshape(batch_size, self.num_heads, seq_len, subspace_dim)\
            .permute(0, 2, 1, 3)\
            .reshape(batch_size, seq_len, embed_dim)

  def forward(X):
    q = self._reshape_to_subspace(q)
    k = self._reshape_to_subspace(k)
    v = self._reshape_to_subspace(v)

    d = q.size(-1)
    scaled_dot_product = q.matmul(k.transpose(-2, -1)) / math.sqrt(d)
    if mask:
      scaled_dot_product.masked_fill(mask[:,:,:seq_len,:seq_len] == 0, float('-inf'))

        


    

In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self):
    super(TransformerBlock, self).__init__()
  def forward(self):
    pass

In [ ]:
class BertModel(nn.Module):
  def __init__(self):
    super(BertModel, self).__init__()
  def forward(self):
    pass